In [1]:
import tqdm
import torch
import torchvision
from torch.utils.data import DataLoader
from PIL import Image
import io

import sys
import os

project_root = os.path.abspath(os.path.join(os.curdir, '../'))
if project_root not in sys.path:
    sys.path.append(project_root)


from common.storage_client import storage_client
from faster_rcnn_dataset import RCNNDataset

In [2]:
DATASET_ID = 1
SET_TYPE_TRAIN = "Training"
SET_TYPE_VALIDATION = "Validation"
SET_TYPE_TEST = "Test"
BATCH_SIZE = 8

NUM_CLASSES = 2
NUM_EPOCHS = 10


In [3]:
def custom_collate(data):
    return data

In [4]:
train_dataset = RCNNDataset(storage_client, DATASET_ID, SET_TYPE_TRAIN)
val_dataset = RCNNDataset(storage_client, DATASET_ID, SET_TYPE_VALIDATION)
test_dataset = RCNNDataset(storage_client, DATASET_ID, SET_TYPE_TEST)

train_data_loader = DataLoader(train_dataset, BATCH_SIZE, True, collate_fn=custom_collate)
val_data_loader = DataLoader(val_dataset, BATCH_SIZE, True, collate_fn=custom_collate)
test_data_loader = DataLoader(test_dataset, BATCH_SIZE, False, collate_fn=custom_collate)

In [5]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights="DEFAULT")
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, NUM_CLASSES)

In [6]:
model

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [7]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cuda


In [8]:
optimizer = torch.optim.SGD(model.parameters(), lr = 0.001, momentum=0.9, weight_decay=0.0005)

In [1]:
model.to(device)
for epoch in range(NUM_EPOCHS):
    print(f"epoch: {epoch}")
    epoch_loss = 0
    for data in tqdm.tqdm(train_data_loader):
        type(data)
        imgs = []
        targets = []
        for d in data:
            imgs.append(d[0].to(device))
            targ = {}
            targ["boxes"] = d[1]["boxes"].to(device)
            targ["labels"] = d[1]["labels"].to(device)
            targets.append(targ)
        
        loss_dict = model(imgs, targets)
        loss = sum(v for v in loss_dict.values())
        epoch_loss += loss.cpu().detach().numpy()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(epoch_loss)

NameError: name 'model' is not defined

In [11]:
torch.save(model, "after_10_ep.pt")

In [ ]:
# public int TrainingDatasetID { get; set; }
# public int StartModelID { get; set; }
# public string Name { get; set; }